# [L Shaped Plots](https://codingcompetitions.withgoogle.com/kickstart/round/0000000000436140/000000000068c509)

- Given
    + A grid `board` size `RxC`
        + `board[r][c]` = `0` or `1`
    + Define a `good segment` as
        + that segment contain only `1`
    + Define an `L-shape` combined by 2 segments as
        + Each of the segments must be a good segment.
        + The two segments must be perpendicular to each other.
        + The segments must share one cell that is an endpoint of both segments.
        + Segments must have length at least 2.
        + The length of the longer segment is twice the length of the shorter segment

- Find the number of `L-shape` in the grid

#### Constraints
- $1 \leq R,C \leq 1000$


# Examples
- valid L-shape

<img src="https://codejam.googleapis.com/dashboard/get_file/AQj_6U1WsahNThgYYtiGIsNwzSCjHVJWsHC6X_ingMAqSrQGYgfbVuL0LVBATISPEtJMRcB--32orWnGB_o38g/examples_correct.png" width="400"/>

- Invalid L-shape

<img src="https://codejam.googleapis.com/dashboard/get_file/AQj_6U2uIrTynEAgxTEUyFr7fXYtrQsMVCBgF3JvvNjteUmCzkLv9kKcp0jGtvdyJxUXG8N3_NZ6pDlWXetE4C6K/examples_incorrect.png" width="400"/>


#### Example 1

```
// Input
4 3
1 0 0
1 0 1
1 0 0
1 1 0

// Output
1

// Explanation
```

<img src="https://codejam.googleapis.com/dashboard/get_file/AQj_6U02RuuoHmgmAvG0ZBiqjEBwJYXaz0T0xEagTAd5cOoedtB-pfxZ399i3flXxnk04O0w1g/sample1.png" width="200"/>


#### Example 2

```
// Input
6 4
1 0 0 0
1 0 0 1
1 1 1 1
1 0 1 0
1 0 1 0
1 1 1 0

// Output
9
```


# Analysis
#### Cache
- Since $1 \leq R,C \leq 1000$, Only an `O(R*C)` solution is viable
    + We need to compute segment length in `O(1)`
    + Idea: prebuild up, down, left, right cache such that
        + `up[r][c]`: length of upward good segment started from `cell[r][c]`
        + `down[r][c]`: length of downward good segment started from `cell[r][c]`
        + `left[r][c]`: length of left good segment started from `cell[r][c]`
        + `right[r][c]`: length of right good segment started from `cell[r][c]`
- Compressed into `length[d][r][c]` = length at r, c, which d =

```
0: up
1: down
2: left
3: right
```

#### Valid L-shape
- Each L-shape formed by:
    + A root cell (`board[r][c]` = 1)
    + A short segment: len >= 2
    + A long segment: len >= 4

- Calculate L-shape based on long and short segment

```
num = min(long_segment/2 - 1, short_segment-1)
```

- Total for each root cell, We have 8 cases to consider. Can simplify into
    + L-shape pos cases
        + Up-Left
        + Up-Right
        + Down-Left
        + Down-Right
    + `f(a,b)`: return number of L-shapes; input: a,b = length of 2 segments
         + case 1: a=long_seg, b=short_seg
         + case 2: b=long_seg, a=short_seg

# Solution

```C++
int R, C;
int board[1005][1005];


int f(int a, int b) {
    int res = 0;
    // Case: a = long seg, b = short_seg
    res += min(a/2 -1, b-1);

    // Case b = long_seg, a = short_seg
    res +=  min(b/2-1, a-1);

    return res < 0 ? 0 : res;
}

int sol() {
    // length[d][r][c] = length at [r][c], which d =
        // 0: up
        // 1: down
        // 2: left
        // 3: right
    int length[4][1005][1005];
    memset(length, 0, sizeof(length));

    // Calc Up
    for(int c=1; c<=C; ++c) {
        int cur = 0;
        for(int r=1; r<=R; ++r) {
            if(board[r][c] == 1) cur += 1;
            else cur = 0;
            length[0][r][c] = cur;
        }
    }

    // Calc Down
    for(int c=1; c<=C; ++c) {
        int cur = 0;
        for(int r=R; r>=1; --r) {
            if(board[r][c] == 1) cur += 1;
            else cur = 0;
            length[1][r][c] = cur;
        }
    }

    // Calc left
    for(int r=1; r<=R; ++r) {
        int cur = 0;
        for(int c=1; c<=C; ++c) {
            if(board[r][c] == 1) cur += 1;
            else cur = 0;
            length[2][r][c] = cur;
        }
    }

    // Calc right
    for(int r=1; r<=R; ++r) {
        int cur = 0;
        for(int c=C; c>=1; --c) {
            if(board[r][c] == 1) cur += 1;
            else cur = 0;
            length[3][r][c] = cur;
        }
    }

    // O(R*C) loop
    int ans = 0;
    for(int r=1; r<=R; ++r) for(int c=1; c<=C; ++c) {
        if(board[r][c] == 0) continue;

        ans += f(length[0][r][c], length[2][r][c]); // Case: Up-left
        ans += f(length[0][r][c], length[3][r][c]); // Case: Up-right
        ans += f(length[1][r][c], length[2][r][c]); // Case: Down-left
        ans += f(length[1][r][c], length[3][r][c]); // Case: Down-Right
    }

    return ans;
}
```